# Real-Time Face Recognition

## First we create the model

In [11]:
import pickle
from math import sqrt
from os import listdir
import face_recognition
from sklearn import neighbors
from face_recognition import face_locations
from os.path import isdir, join, isfile, splitext
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from face_recognition.face_recognition_cli import image_files_in_folder

In [12]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
train_dir = 'faces/train'
test_dir = 'faces/test'

In [13]:
def train(train_dir, model_save_path = "knn_model.sav", n_neighbors = None, knn_algo = 'ball_tree', verbose = False):
    """
    Trains a k-nearest neighbors classifier for face recognition.
    :param train_dir: directory that contains a sub-directory for each known person, with its name.
     (View in source code to see train_dir example tree structure)
     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...
    :param model_save_path: (optional) path to save model of disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified.
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []
    
    print('Processing images for training...')
    for class_dir in listdir(train_dir):
        if not isdir(join(train_dir, class_dir)):
            continue
        for img_path in image_files_in_folder(join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            faces_bboxes = face_locations(image)
            if len(faces_bboxes) != 1:
                if verbose:
                    print("image {} not fit for training: {}".format(img_path, "didn't find a face" if len(faces_bboxes) < 1 else "found more than one face"))
                continue
            X.append(face_recognition.face_encodings(image, known_face_locations = faces_bboxes)[0])
            y.append(class_dir)


    if n_neighbors is None:
        n_neighbors = int(round(sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically as: ", n_neighbors)
    
    print('Training classifier...')
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors = n_neighbors, algorithm = knn_algo, weights = 'distance')
    knn_clf.fit(X, y)
    
    # Save the trained model
    if model_save_path != '':
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)
    
    print('Done')
    
    return knn_clf

### Then we train the model

In [15]:
knn_clf = train(train_dir)

Processing images for training...
Training classifier...
Done


### We can go ahead and test the model

#### 'predict' and 'draw_preds' are just to help test the training model

In [16]:
def predict(X_img_path, knn_clf = None, model_save_path = 'knn_model.sav', DIST_THRESH = 0.5):
    """
    recognizes faces in given image, based on a trained knn classifier
    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. If not specified, model_save_path must be specified.
    :param model_save_path: (optional) path to a pickled knn classifier. If not specified, model_save_path must be knn_clf.
    :param DIST_THRESH: (optional) distance threshold in knn classification. The larger it is, the higher chance of misclassifying an unknown person to a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the 'Unknown' will be passed.
    """

    if not isfile(X_img_path) or splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_save_path == "knn_model.sav":
        raise Exception("Must supply knn classifier either through knn_clf or model_save_path")

    if knn_clf is None:
        with open(model_save_path, 'rb') as f:
            knn_clf = pickle.load(f)

    X_img = face_recognition.load_image_file(X_img_path)
    X_faces_loc = face_locations(X_img)
    if len(X_faces_loc) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_faces_loc)

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors = 1)

    is_recognized = [closest_distances[0][i][0] <= DIST_THRESH for i in range(len(X_faces_loc))]
    
    # predict classes and cull classifications that are not with high confidence
    return [(pred, loc) if rec else ('Unknown', loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_faces_loc, is_recognized)]

In [17]:
def draw_preds(img_path, preds):
    """
    shows the face recognition results visually.
    :param img_path: path to image to be recognized
    :param preds: results of the predict function
    :return:
    """
    source_img = Image.open(img_path).convert("RGBA")
    draw = ImageDraw.Draw(source_img)
    for pred in preds:
        loc = pred[1]
        name = pred[0]
        draw.rectangle(((loc[3], loc[0]), (loc[1],loc[2])), outline="red")
        draw.text((loc[3], loc[0] - 30), name, font = ImageFont.truetype('arial.ttf', 30), fill = (255,255,255,0))
    source_img.show()

In [18]:
# First we load the trained model
knn_clf = pickle.load(open('knn_model.sav', 'rb'))

# Then we test it
for img_path in listdir(test_dir):
    preds = predict(join(test_dir, img_path) ,knn_clf = knn_clf)
    draw_preds(join(test_dir, img_path), preds)

### Finally, we can carry out real-time face recognition

In [19]:
import cv2
import pickle
import numpy as np
from gtts import gTTS
import face_recognition
from sklearn import neighbors
from IPython.display import Audio, display

In [20]:
# This method will greet the one whose face is detected
def say_hi(name):
    
    text_to_speech = gTTS(text = 'hi ' + name, lang = 'en')
    text_to_speech.save('name.mp3')
    display(Audio(filename = 'name.mp3', autoplay = True))

#### And here we go

In [ ]:
# ===== LIGHTS (Initialize some variables) =====
face_locations = []
face_encodings = []
face_names = []
rec_prob = []

welcomed_faces = []

process_this_frame = True

# Load the trained model
knn_clf = pickle.load(open("knn_model.sav", 'rb'))

# ===== CAMERA (Get a reference to webcam #0 (the default one)) =====
cam = cv2.VideoCapture(0)

# ===== AND.... ACTION =====
while True:
    # Grab a single frame of video
    ret, frame = cam.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx = 0.25, fy = 0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        if (len(face_encodings) > 0):

            closest_distances = knn_clf.kneighbors(face_encodings, n_neighbors = 1)

            is_recognized = [closest_distances[0][i][0] <= 0.5 for i in range(len(face_locations))]

            face_names = []
            for pred, prob, loc, rec in zip(knn_clf.predict(face_encodings), knn_clf.predict_proba(face_encodings), face_locations, is_recognized):
                if rec:
                    face_names.append(pred)
                    rec_prob.append('({:.1f}%)'.format(max(np.squeeze(prob))*100))
#                         rec_prob.append(prob)
                else:
                    face_names.append('Unknown')
                    rec_prob.append('')

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name, prob in zip(face_locations, face_names, rec_prob):

        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (180, 100, 0), 2)

        #print(name)
        cv2.putText(frame, '{} {}'.format(name, prob), (left + 6, top), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

        # Say hi
        if name not in welcomed_faces:
            if name != 'Unknown':
                say_hi(name)
                welcomed_faces.append(name)
            else:
                say_hi('')

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'esc' on the keyboard to quit!
    if cv2.waitKey(1) == 27:
        break

# Release webcam
cam.release()
cv2.destroyAllWindows()

### References

- [1] [Face Recognition Based Attendance System](https://github.com/008karan/Face-recognition/blob/master/Attendace%20system.ipynb)